In [1]:
import requests
import os
from dotenv import load_dotenv
from google.cloud import storage

# Setting up the GCP infrastructure

In [2]:
# Load environment variables from .env file
load_dotenv()

# Check if the environment variable is set
if "GOOGLE_APPLICATION_CREDENTIALS" not in os.environ:
    raise ValueError("GOOGLE_APPLICATION_CREDENTIALS not found in environment variables.")

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] 

'credentials/farm-screener-842209070941.json'

In [3]:
#raw_data_bucket = GOOGLE_RAW_DATA_BUCKET 
raw_data_bucket = "farm-screener-raw"

In [4]:
# Create a client
#client = storage.Client(credentials="credentials/farm-screener-842209070941.json")
# Create a client
client = storage.Client()



In [5]:

# List buckets
print("Buckets:")
for bucket in client.list_buckets():
    print(bucket.name)

Buckets:
farm-screener-raw


In [6]:
def upload_to_gcs(data, bucket_name, file_name):
    try:
        client = storage.Client()
        bucket = client.bucket(bucket_name)
        blob = bucket.blob(file_name)
        blob.upload_from_string(json.dumps(data))
        print(f"Data uploaded to GCS: gs://{bucket_name}/{file_name}")
    except Exception as e:
        print(f"Error uploading data to GCS: {e}")

In [7]:
import requests

def download_data(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Failed to download data. Status code: {response.status_code}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"Error downloading data: {e}")
        return None


In [8]:

url = "https://www.hofsuche.schweizerbauern.ch/api/de/farms?offset=24&project=vomhof&size=24"
data = download_data(url)
if data:
    print("Data downloaded successfully!")
    # Now you can work with the downloaded data here
else:
    print("Failed to download data. Please check the URL and try again.")


Data downloaded successfully!


In [9]:
data["hits"][0]

{'_index': 'production-farm-de20240309123004',
 '_id': '14959',
 '_score': 15,
 '_source': {'id': '14959',
  'first_name': 'Sarina',
  'last_name': 'Hofmann-Bötsch',
  'street': 'Hungerbühl 36',
  'zip': '8599',
  'city': 'Salmsach',
  'telephone': '071 463 55 61',
  'mobile': '',
  'email': 'buero@boetsch-gemuese.ch',
  'website': 'https://boetsch-gemuese.ch/hofladen',
  'farmname': 'Hofladen Bötsch',
  'slogan': 'Dich erwartet eine grosse Auswahl an frischem Bötsch-Gemüse, Obst und vielen weiteren Produkten von regionalen Produzenten. Alles was du bei uns einkaufst ist saisonal und regional.\nWir freuen uns auf deinen Besuch!',
  'directions': '',
  'directionspublictransport': '',
  'farmdescription': '',
  'amountofworkers': '',
  'animalsextended': '',
  'culturesextended': '',
  'comment': '',
  'regionalattractions': '',
  'acreage': '',
  'facebook_link': None,
  'instagram_link': 'https://www.instagram.com/hofladen_boetsch/',
  'youtube_link': None,
  'twitter_link': None,
  '